## 분류 미션

1. MNIST 데이터셋으로 분류기를 만들어 테스트 세트에서 97% 정확도를 달성해 보세요.
- 힌트: KNeighborsClassifier가 이 작업에 아주 잘 맞습니다. 좋은 파라미터 값만 찾으면 됩니다.

In [1]:
# import matplotlib.pyplot as plt
# import koreanize_matplotlib
# %matplotlib inline

### sklearn 데이터셋에서 'Toy Dataset'인 'mnist_784'를 불러온다

In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784',version=1,as_frame=False,parser='auto')

### mnist_784에 data 값을 x에 target값을 y에 저장한다.

In [3]:
x,y = mnist['data'],mnist['target']

In [4]:
# 그리고 이때 target 값이 object로 되어있으니, 정수로 데어터 타입을 바꿔준다.
import numpy as np
y=y.astype(np.uint8)
y

array([5, 0, 4, ..., 4, 5, 6], dtype=uint8)

### 훈련 세트와 데이터 세트로 나누어 준다.

In [5]:
x_train, x_test, y_train, y_test = x[:60000], x[60000:], y[:60000],y[60000:]
x_train.shape

(60000, 784)

### 7과 7이 아닌 이진 분류기 훈련을 위해 타겟 값을 변경한다.

In [6]:
y_train_7 = (y_train==7)
y_test_7 = (y_test==7)

### sklearn의 KneighborsClassifier 분류기를 불러오고 학습 시킨다.

In [7]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier()
knn_clf.fit(x_train,y_train_7)

KNeighborsClassifier()

### cross_val_score를 이용하여 해당 분류기의 정확도 성능을 평가한다.

In [8]:
from sklearn.model_selection import cross_val_score
cross_val_score(knn_clf,x_train,y_train_7,cv=3,scoring='accuracy',verbose=1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.2min finished


array([0.99425, 0.9923 , 0.9928 ])

2. MNIST 이미지를 (왼, 오른, 위, 아래) 어느 방향으로든 한 픽셀 이동시킬 수 있는 함수를 만들어보세요. 그런 다음 훈련 세트에 있는 각 이미지에 대해 네 개의 이동된 복사본(방향마다 한 개씩)을 만들어 훈련 세트에 추가하세요. 마지막으로 이 확장된 데이터셋에서 앞에서 찾은 최선의 모델을 훈련시키고 테스트 세트에서 정확도를 측정해보세요. 모델 성능이 더 높아졌는지 확인해보세요. 인위적으로 훈련 세트를 늘리는 이 기법을 데이터 증식 또는 훈련 세트 확장이라고 합니다.

In [9]:
# 우선 mnsit_784의 데이터들으 보기 쉽게 28행, 28열로 재배열해 출력해보자.
np.set_printoptions(linewidth=np.inf)
some_digit_image = x[0].reshape(28,28)
some_digit_image


array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 170, 253, 253, 253, 253, 253, 225, 172

### 위 출력된 2차원 배열을 멀리서 보면 숫자 5처럼 보인다.

### 위 출력된 모양에서 상,하,좌,우 로 한칸씩 움직여 보자.

In [10]:
# 왼쪽으로옮기기
# 숫자 5처럼 보여지는 배열이 왼쪽으로 이동한 것 처럼 보여야 하기 때문에
# numpy.c_를 이용하여 컬럼 별로 0으로 이루어진 배열을 column 으로 합한다음
# numpy.delete로 각 행의 첫번째 요소를 삭제시켜준다
move = np.zeros(28).astype(np.uint8) # 훈련 셋에 추가시킬 0으로 이루어진 배열이다.
np.delete(np.c_[some_digit_image,move],0,axis=1)

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 170, 253, 253, 253, 253, 253, 225, 172, 253

#### 마치 왼쪽으로 한칸 이동한 것 처럼 보인다.

In [11]:
# 위의 방법과 마찬가지로 오른쪽으로도 이동 시켜 보자.
# 0으로 이루어진 배열과 삭제할 배열의 위치를 반대로 하면 된다.
np.delete(np.c_[move,some_digit_image],-1,axis=1)

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 170, 253, 253, 253, 253, 253, 225

### 마치 오른쪽으로 한칸 이동한 것과 같다.

### 위와 동일한 방법으로 아래, 위로 한칸 이동시키는 move_up과 move_down을 만든 후,
### move_lrud()라는 함수를 만들어 상,하,좌,우 모두 이동시킨되 나온 배열을 하나로 합쳐주자.

In [12]:
def move_lrud(x):
    x_digit = x.reshape(28,28)
    move = np.zeros(28).astype(np.uint8)
    move_dim_2 = np.expand_dims(move,axis=0)
    move_up = np.delete(np.r_[x_digit,move_dim_2],0,axis=0).reshape(-1,)
    move_down = np.delete(np.r_[move_dim_2,x_digit],-1,axis=0).reshape(-1,)
    move_left = np.delete(np.c_[x_digit,move],0,axis=1).reshape(-1,)
    move_right = np.delete(np.c_[move,x_digit],-1,axis=1).reshape(-1,)
    move_lrud_result = np.concatenate((move_up,move_down,move_left,move_right),axis=0)
    return move_lrud_result

### move_lrud() 함수로 x의 첫번째 배열을 실행 시킨후 reshape으로 열만 28로 맞춘 후 행은 알아서 맞춰지게 출력해보자
### 숫자 5형태의 배열이 상,하,좌,우 이동된 배열을 볼 수 있다.

In [13]:
np.set_printoptions(linewidth=np.inf,threshold=np.inf) # 이 코드는 출력될 값의 생략부분을 없애주는 코드이다.
move_lrud(x[0]).reshape(-1,28)

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 170, 253, 253, 253, 253, 253, 225, 172, 253, 242, 195,  64,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,  49, 238, 253, 253, 253, 253, 253, 253, 253, 253, 251,  93,  82

### 그 후 새 리스트를 만들어 반복문으로 x_train의 모든 데이터 셋을 move_lrud()함수로 실행시킨다.
### 나온 모든 결과 값을 L이라는 리스트에 append 시킨다.
### 그 후 리스트를 numpy배열로 변경 시킨다.

In [14]:
L = []
for i in range(len(x_train)):
    N = move_lrud(x[i])
    L.append(N)
x_train_move = np.array(L)

### x_train_move라는 데이터셋이 만들어 졌다.

In [15]:
# target이 될 y_train셋도 상,하,좌,우 총 4배씩 늘어난 data 데이터 셋에 맞춰 각 요소를 4배씩 늘려준다.
y_train_repeat = np.repeat(y_train,4)
y_train_repeat.shape

(240000,)

In [ ]:
# 학습모델에 학습시킬 두 데이터의 모양이 다르다.
# 이는 x_train_move의 열을 784로 만듦으로써 해결할 수있다.
x_train_move.shape,y_train_repeat.shape

((60000, 3136), (240000,))

In [ ]:
x_train_move = x_train_move.reshape(-1,784)
x_train_move.shape,y_train_repeat.shape
# 다시 비교해보니 같은 행을 가진 두개의 데이터 셋이 완성되었다.

((240000, 784), (240000,))

In [ ]:
# 다시 학습모델에 학습시켜준다.
knn_clf = KNeighborsClassifier()
knn_clf.fit(x_train_move,y_train_repeat)

KNeighborsClassifier()

In [ ]:
# cross_val_score를 이용하여 해당 분류기의 정확도 성능을 평가한다.
cross_val_score(knn_clf,x_train_move,y_train_repeat,cv=3,scoring='accuracy',verbose=1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 20.9min finished


array([0.96765  , 0.9677125, 0.9686   ])